## Module is used to compare legacy outputs from pix-plot to clip-plot.  

The goal is catch unforeseen changes that modify the baseline outputs

In [ ]:
#|default_exp compare

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#|hide
from nbdev.showdoc import *

In [ ]:
#|export
import filecmp
import shutil
import os
from pathlib import Path
import difflib
import json
from typing import Optional, List, Union, Tuple

from clip_plot import utils, clip_plot

In [ ]:
#| export
basedir = clip_plot.get_clip_plot_root()
clip_plt_dir = basedir / "tests/smithsonian_butterflies_10/output_test_temp"
pix_plt_dir = basedir / "tests/butterflies_baseline"
# pix_plt_dir = basedir / "output"


In [ ]:
#| export

MANIFEST1 = "data/manifests/manifest-test_diff.json"
MANIFEST2 = "data/manifest.json"

# Files to track and compare
COMPARE_FILES = [
    "data/atlases/test_diff/atlas_positions.json",
    "data/hotspots/hotspot-test_diff.json",
    "data/imagelists/imagelist-test_diff.json",

    # Layouts
    "data/layouts/umap-jittered-test_diff.json",
    "data/layouts/grid-test_diff.json",
    "data/layouts/rasterfairy-test_diff.json",
    "data/layouts/categorical-labels-test_diff.json",
    "data/layouts/categorical-test_diff.json",
    "data/layouts/timeline-test_diff.json",
    "data/layouts/timeline-labels-test_diff.json",
    "data/layouts/umap-test_diff.json",

    # Manifest
    MANIFEST1,
    MANIFEST2,

    # Metadata (per image)
    "data/metadata/file/329c2b4d5-8137-414b-b98e-ff04907e8ea6.jpg.json",
    "data/metadata/file/0dae7d86-9c14-11ed-a00f-a37ce258aeb3.jpg.json",
    "data/metadata/file/9fea3150-a3d4-11ed-aeea-e36f1256f233.jpg.json",
    "data/metadata/file/3fee89f9b-ba5c-4f2e-8532-6e390e2cf0c9.jpg.json",
    "data/metadata/file/31fd87d81-7ff5-4e50-b718-0d4a259d47c1.jpg.json",
    "data/metadata/file/36829498c-0eda-4c84-b89d-c893c75cfa68.jpg.json",
    "data/metadata/file/354747d21-638c-4e23-a655-c0bb4de18941.jpg.json",
    "data/metadata/file/376dd3835-ea28-48ab-b7e8-c7c929a77a01.jpg.json",
    "data/metadata/file/30aeb051d-ee0d-4c5b-8a85-a8da7baef5fd.jpg.json",
    "data/metadata/file/3c3407493-e0d9-43fe-a2a2-7c43395c90c5.jpg.json",
    "data/metadata/file/329a4c094-8536-4396-be70-3d9b5d0744d9.jpg.json",
    "data/metadata/file/386168016-7276-4b02-b713-5f36ac2ef452.jpg.json",

    # Metadata
    "data/metadata/filters/filters.json",
    "data/metadata/options/Beautiful.json",
    "data/metadata/options/Tiny.json",
    "data/metadata/options/Spots.json",
    "data/metadata/options/Colorful.json",
    "data/metadata/options/Broken.json",
    "data/metadata/options/Gray.json",
    "data/metadata/options/Ugly.json",
    "data/metadata/options/Moth.json",
    "data/metadata/dates.json",    
]

In [ ]:
#| export
def log_output(txt, space=0):
    # Hook for logging results
    if isinstance(txt,str):
        print(f'{" " * space}{txt}')
    else:
        print("".join(txt))


def compare_files_similar_text(file1: str, file2:str) -> Tuple[bool,str]:
    """    This function compares two files to check if they are the same,
    
    Args:
        file1 (str): path for file 1
        file2 (str): path for file 2

    Returns:
        tuple(bool, str):
            bool: true is files are the same
            str: feedback if file difference
    """
    try:
        if filecmp.cmp(file1, file2) is False:
            if os.path.isfile(file1) and os.path.isfile(file2):
                with open(file1, 'r') as f1, open(file2, 'r') as f2:
                    differ = difflib.Differ()
                    file1_lines = f1.readlines()
                    file2_lines = f2.readlines()
                    diff = list(differ.compare(file1_lines, file2_lines))
                    return False, diff
        else:
            return True, ""
    except FileNotFoundError:
        return False, "Could not open files"


def compare_outputs():
    """Function loops named files in the COMPARE_FILES
    list to compare the files form the legacy output and
    the new output.  
    """
    for j, file in enumerate(COMPARE_FILES):
        log_output(f"\n#{j} Comparing {file}")
        file_clip = clip_plt_dir / file
        file_pix = pix_plt_dir / file
        chk, txt = compare_files_similar_text(file_clip, file_pix)
        if chk is False:
            log_output(f'- Fail', 2)
            log_output(txt, 4)
        else:
            log_output(f'+ Identical', 2)


In [ ]:
#| export
def comFile():
    """Compare two directories

    Call out missing files and folders and files with
    different content.
    """
    fail = False
    log_output("Comparing Files and folders")
    fileComp = filecmp.dircmp(clip_plt_dir, pix_plt_dir)

    # Create a queue
    currs = [fileComp]
    while currs:
        curr = currs.pop()

        try:
            # Check for different files
            if curr.diff_files:
                loc = str(curr.right).replace(str(pix_plt_dir),"")
                log_output(f'Different at {loc}:\n    {curr.diff_files}\n',2)
                fail = True
            for k, v in curr.subdirs.items():
                currs.append(v)

            # Check if pix_plt_dir is missing files/folders
            for asset in curr.left_list:
                if asset not in curr.right_list:
                    fail = True
                    log_output(f'Pix-plot Missing: {asset}',2)

            # Check if clip_plt_dir is missing files/folders
            for asset in curr.right_list:
                if asset not in curr.left_list:
                    fail = True
                    log_output(f'Clip-plot Missing: {asset}',2)

        except FileNotFoundError as e:
            msg = f'{str(e)} \n   Check directories path for clip_plt_dir and pix_plt_dir!'
            log_output(msg,2)

    if fail is False:
        log_output("No difference found!")

    return fail

In [ ]:
#| export
def replace_and_save(file):
    """Functions replaces creation date and
    replaces directory references to match clip_plot
    """
    with open(clip_plt_dir / file) as f:
        ogData = json.load(f)
        date = ogData['creation_date']
    
    with open(pix_plt_dir / file) as f:
        newData = json.load(f)
        newData['creation_date'] = date

    with open(pix_plt_dir / file, "w") as out:
        json.dump(newData, out, indent=4)

    # Open the file in read mode
    with open(pix_plt_dir / file, 'r') as read:
        # Read the contents of the file
        file_contents = read.read()

        # Replace the old text with the new text
        new_contents = file_contents.replace(str(pix_plt_dir), str(clip_plt_dir))

    with open(pix_plt_dir / file, 'w') as out:
        out.write(new_contents)


def fix_expected_diff():
    """Update pix_plot manifest to match clip_plot manifest

    Updates only:
        Creation date
        Directory references 
    """
    # Update date
    replace_and_save(MANIFEST1)
    replace_and_save(MANIFEST2)

In [ ]:
#| export
if __name__ == "__main__":
    fix_expected_diff()
    comFile()
    # compare_outputs()

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()